In [1]:
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)
import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn import linear_model
import pickle
import subprocess
import networkx as nx
# import scipy.stats as stats

from Bio import SeqIO
from Bio import SeqFeature as sf
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq

import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')

#New here
from ete3 import Tree

what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
Added C:\Users\heine\github\expression_broad_data to path: 
['', 'C:\\Users\\heine\\Anaconda3\\python36.zip', 'C:\\Users\\heine\\Anaconda3\\DLLs', 'C:\\Users\\heine\\Anaconda3\\lib', 'C:\\Users\\heine\\Anaconda3', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Babel-2.5.0-py3.6.egg', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\heine\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\heine\\.ipython', 'C:\\Users\\heine\\github\\expression_broad_data']
I am about to import a library
I am importing io_library
what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
data process

In [2]:
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#Load KL SC PKA data
fname_cross_spec_comparison = data_processing_dir + os.path.normpath("20170817_klscpka/cross_species_compare_ASminus_ASplus_YPD.csv")
kl_sc_PKA_data = pd.read_csv(fname_cross_spec_comparison, index_col=0)


#Total number of KL genes that have a paralog in SCER
# kl_sc_PKA_data_paralogs = kl_sc_PKA_data[kl_sc_PKA_data["N SCer Orthologs"]==2]

# print('Total number of KL genes that map to a set of paralogs in S.Cer: ' + str(int(len(kl_sc_PKA_data_paralogs)/2)))

# #Total of these that are WGD paralogs
# ohnolog_set = set(ohnologs['Gene 1']) | set(ohnologs['Gene 2'])

# non_ohnologs = set(kl_sc_PKA_data_paralogs['sc_genename']) - ohnolog_set
# ohnologs_klac_orth = set(kl_sc_PKA_data_paralogs['sc_genename']) & ohnolog_set

# print('KL genes that map to a set of Ohnologs: ' + str(int(len(ohnologs_klac_orth)/2)) + '\n' +
#      'KL genes that map to a non Ohnolog paralog: ' + str(int(len(non_ohnologs))))


In [31]:
#Build dataframe that finds topology in best tree for all SC Genes

#Make Dictionary for SC genes out of all_id_conversion.txt file
io_library.build_phylo_conversion_sc()

I am importing io_library
what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
data processing dir is C:\Users\heine\github\expression_broad_data\expression_data\
no ensemble id for Phy001SPPJ_YEAST, using genename: ATP22
no ensemble id for Phy001SPO1_YEAST, using genename: FLO8
no ensemble id for Phy001SPPW_YEAST, using genename: TY3B-I
no ensemble id for Phy001SPND_YEAST, using genename: AQY2
no ensemble id or genename for Phy00396QV_YEAST, using phylome DB id
no ensemble id for Phy001SPPK_YEAST, using genename: TCM62
no ensemble id for Phy001SPQ1_YEAST, using genename: DYN3
no ensemble id for Phy001SPOV_YEAST, using genename: OCT1
no ensemble id for Phy001SPOB_YEAST, using genename: HXT4
no ensemble id or genename for Phy00396QW_YEAST, using phylome DB id
no ensemble id for Phy001SPO9_YEAST, using genename: HFA1
no ensemble id for Phy001SPOK_YEAST, using genename: MTG2
no ensemble id for Phy001SPNQ_YEAST, usi

In [3]:
sc_phylomedb_convert = pd.read_table(data_processing_dir + os.path.normpath("phylogenetic_trees/sc_id_conversion.txt"), header=None)
sc_phylomedb_convert.columns = ["phylomedb_name","sc_genename"]

trees = pd.read_table(data_processing_dir + os.path.normpath("phylogenetic_trees/best_trees.txt"), header = None)
trees.columns = ["phylomedb_name","alignment_method","score","tree"]
trees.set_index("phylomedb_name", inplace=True)



In [6]:
#check SC genename for give philome name
gois = ['Phy000CZF5_YEAST', 'Phy000CX7L_YEAST', 'Phy000CZ7U_YEAST']
io_library.SC_common_name_lookup(list(sc_phylomedb_convert[sc_phylomedb_convert["phylomedb_name"].isin(gois)]['sc_genename']))

['SNZ1', 'SNZ2', 'SNZ3']

['SNZ1', 'SNZ2', 'SNZ3'] - these genes have multiple orthologs in SC

In [4]:
#gene = sc_phylomedb_convert.iloc[0]["phylomedb_name"]
#gene = "Phy000CYB4_YEAST"

#Make this into a function


KLE_branch_seeds = ["KLULA", "ASHGO", "KLUWA", "LACTH","SACKL"]
ZT_branch_seed = ["TORDC","ZYGRO"] 
WGH_branch_seed = ["YEAST","SACCA", "VANPO"]       
#51660: N. Bacillosporus, 51914: C. CAstelii, K.Nag: 588726, N. Dairenensis: NAUDC, 
#1071379: Tet Blatt, TETPH: Tet. Phaffii, KAZAF: K. Africana, CANGA: Candida glabrata, SACBA
outgroups = ["CANAL","YARLI","DEKBR"]

topologies = io_library.assign_topologies(trees, WGH_branch_seed, KLE_branch_seed, ZT_branch_seed, outgroups)

trees['topologies_{}_{}_{}'.format(WGH_branch_seed,KLE_branch_seed,ZT_branch_seed)] = topologies



#try for both T and Z


Phy000CX7L_YEAST has more than 2 paralogs
['Phy004FX96_TORDC', 'Phy0008PLP_KLULA', 'Phy000CZF5_YEAST', 'Phy000CX7L_YEAST', 'Phy000CZ7U_YEAST']
Threw out Phy004FVT2_TORDC because outgroup gene between it and seed
Threw out Phy0008NP3_KLULA because outgroup gene between it and seed
Threw out Phy0008OXE_KLULA because outgroup gene between it and seed
Threw out Phy004FXU4_TORDC because outgroup gene between it and seed
Threw out Phy004FYW1_TORDC because outgroup gene between it and seed
Threw out Phy004FYK9_TORDC because outgroup gene between it and seed
Threw out Phy004FW43_TORDC because outgroup gene between it and seed
Threw out Phy004FV9Y_TORDC because outgroup gene between it and seed
non-standard phylogeny for Phy000CXLE_YEAST, leaf subset =
['Phy000CYQS_YEAST', 'Phy000CYO8_YEAST', 'Phy000CXLE_YEAST', 'Phy004FVYS_TORDC', 'Phy0008M5J_KLULA', 'Phy0008MKM_KLULA', 'Phy000CW8C_YEAST', 'Phy004FWCD_TORDC', 'Phy000CWM3_YEAST', 'Phy000CX09_YEAST', 'Phy000CZ87_YEAST']
Threw out Phy0008M6G_KLUL

Threw out Phy0008NFG_KLULA because outgroup gene between it and seed
Threw out Phy0008O9X_KLULA because outgroup gene between it and seed
Threw out Phy0008OH1_KLULA because outgroup gene between it and seed
Threw out Phy004FVBQ_TORDC because outgroup gene between it and seed
Phy000CXGF_YEAST missing TORDC and KLULA protein
['Phy000CXGF_YEAST']
Phy000CZO6_YEAST missing TORDC and KLULA protein
['Phy000CWPO_YEAST', 'Phy000CZDT_YEAST', 'Phy000CVQY_YEAST', 'Phy000CZO6_YEAST', 'Phy000CWTA_YEAST']
Threw out Phy0008N6P_KLULA because outgroup gene between it and seed
Threw out Phy0008O77_KLULA because outgroup gene between it and seed
Threw out Phy000PHQX_KLULA because outgroup gene between it and seed
Threw out Phy0008M3B_KLULA because outgroup gene between it and seed
Threw out Phy0008PUM_KLULA because outgroup gene between it and seed
Alert threw out both KLULA genes for seed: Phy000CXVE_YEAST
Threw out Phy004FVHE_TORDC because outgroup gene between it and seed
Threw out Phy004FVH1_TORDC bec

non-standard phylogeny for Phy000CX2Z_YEAST, leaf subset =
['Phy004FVR5_TORDC', 'Phy000NLT3_YEAST', 'Phy000CX2Z_YEAST', 'Phy004FYQI_TORDC', 'Phy0008PV9_KLULA', 'Phy000CXUQ_YEAST', 'Phy000CXUP_YEAST']
Threw out Phy0008NJ7_KLULA because outgroup gene between it and seed
Threw out Phy0008OSS_KLULA because outgroup gene between it and seed
Threw out Phy004FYY9_TORDC because outgroup gene between it and seed
Threw out Phy004FY3K_TORDC because outgroup gene between it and seed
Phy000CX4W_YEAST has more than 2 paralogs
['Phy000CVMG_YEAST', 'Phy000CX4W_YEAST', 'Phy004FX0X_TORDC', 'Phy0008NKV_KLULA', 'Phy000CWTH_YEAST', 'Phy000CWYU_YEAST']
Phy000CWUX_YEAST missing KLULA protein
['Phy004FWK7_TORDC', 'Phy000CWUX_YEAST']
Threw out Phy0008PW7_KLULA because outgroup gene between it and seed
Threw out Phy004FX20_TORDC because outgroup gene between it and seed
Phy000CXP7_YEAST has more than 2 paralogs
['Phy000CYOC_YEAST', 'Phy000CXP7_YEAST', 'Phy004FVA1_TORDC', 'Phy0008NB1_KLULA', 'Phy000CXT5_YEAST']


Threw out Phy004FVXA_TORDC because outgroup gene between it and seed
Threw out Phy004FYDI_TORDC because outgroup gene between it and seed
Threw out Phy004FVR3_TORDC because outgroup gene between it and seed
Threw out Phy004FZ0J_TORDC because outgroup gene between it and seed
Threw out Phy004FY1N_TORDC because outgroup gene between it and seed
Threw out Phy004FYKE_TORDC because outgroup gene between it and seed
non-standard phylogeny for Phy000CVY7_YEAST, leaf subset =
['Phy000CXWI_YEAST', 'Phy000CW4Z_YEAST', 'Phy000CW6N_YEAST', 'Phy003FL7E_KLULA', 'Phy0008PXM_KLULA', 'Phy004FW9D_TORDC', 'Phy004FVKH_TORDC', 'Phy000CXRD_YEAST', 'Phy000CW3H_YEAST', 'Phy000CYBG_YEAST', 'Phy000CVY7_YEAST']
Phy000CY7M_YEAST missing KLULA protein
['Phy004FX0Y_TORDC', 'Phy000CY7M_YEAST']
Threw out Phy0008OLT_KLULA because outgroup gene between it and seed
Threw out Phy0008MSF_KLULA because outgroup gene between it and seed
Threw out Phy004FYUY_TORDC because outgroup gene between it and seed
Threw out Phy004FWJ

Alert threw out both TORDC genes for seed: Phy000CWR8_YEAST
Phy000CWR8_YEAST missing TORDC protein
['Phy0008M3X_KLULA', 'Phy000CWR8_YEAST']
Phy000CX6X_YEAST missing KLULA protein
['Phy004FY9S_TORDC', 'Phy000CX6X_YEAST']
non-standard phylogeny for Phy000CWKV_YEAST, leaf subset =
['Phy000CY0P_YEAST', 'Phy000CYP9_YEAST', 'Phy004FYTM_TORDC', 'Phy000CWKW_YEAST', 'Phy000CWKV_YEAST', 'Phy001SPOB_YEAST', 'Phy000CZRB_YEAST', 'Phy000CY0K_YEAST', 'Phy004FX68_TORDC', 'Phy004FXSX_TORDC', 'Phy004FVXA_TORDC', 'Phy000CX94_YEAST', 'Phy000CZ5A_YEAST', 'Phy004FXNW_TORDC', 'Phy000CXWG_YEAST', 'Phy000CWKY_YEAST', 'Phy000CXWI_YEAST', 'Phy004FYDI_TORDC', 'Phy0008M68_KLULA', 'Phy000CZQT_YEAST', 'Phy000CWYP_YEAST']
Threw out Phy0008OH1_KLULA because outgroup gene between it and seed
Threw out Phy0008O8T_KLULA because outgroup gene between it and seed
Threw out Phy0008N0W_KLULA because outgroup gene between it and seed
Threw out Phy0008MH4_KLULA because outgroup gene between it and seed
Threw out Phy003FL58_KLU

...

Threw out Phy004FWKG_TORDC because outgroup gene between it and seed
Threw out Phy004FX4G_TORDC because outgroup gene between it and seed
Threw out Phy004FXJX_TORDC because outgroup gene between it and seed
Alert threw out both TORDC genes for seed: Phy000CZR9_YEAST
Phy000CZR9_YEAST missing TORDC and KLULA protein
['Phy000CXRY_YEAST', 'Phy000CXRQ_YEAST', 'Phy000CVYE_YEAST', 'Phy000CYL2_YEAST', 'Phy000CWYT_YEAST', 'Phy000CYL1_YEAST', 'Phy000D0LG_YEAST', 'Phy000CZR9_YEAST']
Phy000CY0G_YEAST missing TORDC and KLULA protein
['Phy000CVLP_YEAST', 'Phy000CY0D_YEAST', 'Phy000CWQV_YEAST', 'Phy000D0LM_YEAST', 'Phy000CX79_YEAST', 'Phy000CZF0_YEAST', 'Phy000CXRK_YEAST', 'Phy003G9O8_YEAST', 'Phy003G9OA_YEAST', 'Phy000CZ0Y_YEAST', 'Phy000CZ0V_YEAST', 'Phy000CWQX_YEAST', 'Phy000CY0G_YEAST', 'Phy000CWYE_YEAST']
Threw out Phy0008MU3_KLULA because outgroup gene between it and seed
Threw out Phy003FL73_KLULA because outgroup gene between it and seed
Alert threw out both KLULA genes for seed: Phy000CZUH_Y

Phy000CWR5_YEAST missing TORDC and KLULA protein
['Phy000CWR5_YEAST']
Threw out Phy0008P4Q_KLULA because outgroup gene between it and seed
Threw out Phy0008LZV_KLULA because outgroup gene between it and seed
Threw out Phy0008OV8_KLULA because outgroup gene between it and seed
Threw out Phy004FW8Z_TORDC because outgroup gene between it and seed
Threw out Phy004FYDF_TORDC because outgroup gene between it and seed
Threw out Phy004FXXM_TORDC because outgroup gene between it and seed
Phy000CXJB_YEAST has more than 2 paralogs
['Phy000CZ03_YEAST', 'Phy000CZRR_YEAST', 'Phy000CXJB_YEAST', 'Phy004FXWV_TORDC', 'Phy0008MH0_KLULA']
Phy000CW3Q_YEAST missing TORDC protein
['Phy0008NRX_KLULA', 'Phy000CW3Q_YEAST']
Phy000CZWK_YEAST missing KLULA protein
['Phy004FW6G_TORDC', 'Phy000CZWK_YEAST']
Threw out Phy0008OKZ_KLULA because outgroup gene between it and seed
Threw out Phy0008OJL_KLULA because outgroup gene between it and seed
Alert threw out both KLULA genes for seed: Phy000D0AZ_YEAST
Phy000D0AZ_YEAS

Threw out Phy0008PF8_KLULA because outgroup gene between it and seed
Threw out Phy004FX93_TORDC because outgroup gene between it and seed
Threw out Phy004FZ0Z_TORDC because outgroup gene between it and seed
Phy000CZQD_YEAST has more than 2 paralogs
['Phy000CXGJ_YEAST', 'Phy0008P9S_KLULA', 'Phy000CZQD_YEAST', 'Phy004FX3D_TORDC', 'Phy000CYCX_YEAST']
Phy000CZKM_YEAST missing TORDC protein
['Phy000CZKM_YEAST', 'Phy0008N44_KLULA']
Phy000CWYO_YEAST missing KLULA protein
['Phy004FY8Z_TORDC', 'Phy000CWYO_YEAST']
Threw out Phy0008M2N_KLULA because outgroup gene between it and seed
Threw out Phy0008O65_KLULA because outgroup gene between it and seed
Threw out Phy0008OTF_KLULA because outgroup gene between it and seed
Alert threw out both KLULA genes for seed: Phy000CWPN_YEAST
Threw out Phy004FW9Q_TORDC because outgroup gene between it and seed
Threw out Phy004FYFK_TORDC because outgroup gene between it and seed
Threw out Phy004FXZ5_TORDC because outgroup gene between it and seed
Phy000CWPN_YEAST

Phy000CVYZ_YEAST missing TORDC and KLULA protein
['Phy003ETYP_YEAST', 'Phy000CWC9_YEAST', 'Phy000CWQ4_YEAST', 'Phy000CVYZ_YEAST']
non-standard phylogeny for Phy000CX73_YEAST, leaf subset =
['Phy000CYOA_YEAST', 'Phy000CXP6_YEAST', 'Phy004FY1R_TORDC', 'Phy000CX73_YEAST', 'Phy0008N20_KLULA', 'Phy0008MLG_KLULA']
Phy000CXJK_YEAST missing TORDC protein
['Phy000CVPK_YEAST', 'Phy000D004_YEAST', 'Phy000CWGX_YEAST', 'Phy000CY08_YEAST', 'Phy000CZ3N_YEAST', 'Phy000CWK1_YEAST', 'Phy000CWDJ_YEAST', 'Phy000CXK0_YEAST', 'Phy000CY7N_YEAST', 'Phy000CYUH_YEAST', 'Phy000CWLM_YEAST', 'Phy000CZSS_YEAST', 'Phy000D0K6_YEAST', 'Phy000CWIH_YEAST', 'Phy000CY7Q_YEAST', 'Phy000CXJK_YEAST', 'Phy000CVKT_YEAST', 'Phy000CZN7_YEAST', 'Phy000CVP2_YEAST', 'Phy000CZ6D_YEAST', 'Phy000CWFL_YEAST', 'Phy000CZGJ_YEAST', 'Phy000CYZ9_YEAST', 'Phy000CWIE_YEAST', 'Phy000CX9E_YEAST', 'Phy000CVZT_YEAST', 'Phy000CVM3_YEAST', 'Phy000D064_YEAST', 'Phy000CWGV_YEAST', 'Phy000CWBM_YEAST', 'Phy000D01O_YEAST', 'Phy003FL70_KLULA']
Threw out 

Threw out Phy0008O1H_KLULA because outgroup gene between it and seed
Threw out Phy0008N9U_KLULA because outgroup gene between it and seed
Threw out Phy0008MOX_KLULA because outgroup gene between it and seed
Threw out Phy004FVK3_TORDC because outgroup gene between it and seed
Threw out Phy004FWDI_TORDC because outgroup gene between it and seed
Threw out Phy004FW96_TORDC because outgroup gene between it and seed
Threw out Phy004FW02_TORDC because outgroup gene between it and seed
Threw out Phy004FYC0_TORDC because outgroup gene between it and seed
Phy000CZA2_YEAST has more than 2 paralogs
['Phy000CX3D_YEAST', 'Phy000D06Z_YEAST', 'Phy000D0DX_YEAST', 'Phy000CZA3_YEAST', 'Phy000CZA2_YEAST', 'Phy004FYAZ_TORDC', 'Phy0008O1I_KLULA', 'Phy000CVPD_YEAST']
Threw out Phy0008P2K_KLULA because outgroup gene between it and seed
Threw out Phy0008P2N_KLULA because outgroup gene between it and seed
Threw out Phy0008P2M_KLULA because outgroup gene between it and seed
Alert threw out both KLULA genes for s

In [5]:
trees

,alignment_method,score,tree,topologies_YEAST_KLULA_TORDC
phylomedb_name,,,,
Phy000CX7L_YEAST,LG,-5817.18,"((((((((((((Phy003LSK4_51914:0.312411,(Phy003M...",C
Phy000CZXQ_YEAST,LG,-8578.15,"(((((((Phy00244Z8_ZYGRO:0.858865,(Phy004F982_1...",A
Phy000CZPL_YEAST,LG,-23420.60,"(((((((((((Phy000NQML_SACCA:0.269618,Phy004FK9...",A
Phy000CXN0_YEAST,LG,-17546.60,"((((((((((((Phy004FLWI_NAUDC:0.455222,Phy000NS...",A
Phy000D0GQ_YEAST,VT,-10982.60,"(((((((((((Phy003ETPM_SACBA:0.117642,Phy000D0G...",C
Phy000CY1N_YEAST,JTT,-28090.20,"(((((Phy003FK4B_ASHGO:1.68388,Phy0008PFP_KLULA...",C
Phy000CY4R_YEAST,JTT,-36450.20,"(((((((Phy000NO0U_SACBA:0.170305,Phy000CY4R_YE...",A
Phy000CVKG_YEAST,LG,-2996.59,"(((((Phy0008MTI_KLULA:0.982159,Phy004FVNO_TORD...",C
Phy000CXLE_YEAST,LG,-94834.30,(((((((((((((((((((((((Phy004F1ZJ_588726:0.274...,under construction


In [181]:
common_name = 'MPC2'
tree = Tree(trees_subset[common_name])
gene = phylomedb_names[phylomedb_names['SC_common_name']==common_name]['phylomedb_name'].iloc[0]
# print(tree)

#Extract only leaves that are are _YEAST, _KLULA, ZT_branch seed (i.e. _TORDC, _ZYGRO)
leaf_names = tree.get_leaf_names()
leaf_subset = [name for name in leaf_names if name.split('_')[1] in {"YEAST","KLULA",ZT_branch_seed}]

#Check for 1-2 S.Cer proteins and only 1 each of KL and ST branch
leaf_subset_spec = [name.split("_")[1] for name in leaf_subset]
leaf_subset_spec_counter = Counter(leaf_subset_spec)

leaf_subset_spec_counter

for WGH_branch in ['KLULA',ZT_branch_seed]:
    if (leaf_subset_spec_counter[WGH_branch]>1):
        #Throw out extra WGH branch gene if children of common ancestor between it and current gene are in an outgroup.  
        
        #Prune down tree to include spec of interest and outgroups
        leaf_subset_outgroups = [name for name in leaf_names if name.split('_')[1] in set(["YEAST","KLULA",ZT_branch_seed] + outgroups)]
        tree.prune(leaf_subset_outgroups) 
        #print(tree)
        #for each of the branch genes. 
        branch_genes = [leaf for ind,leaf in enumerate(leaf_subset) if leaf_subset_spec[ind]==WGH_branch]
        for branch_gene in branch_genes:
            ancestor = tree.get_common_ancestor([gene,branch_gene])
            ancestor_leaves = ancestor.get_leaf_names()
            ancestor_leaves_spec = [name.split("_")[1] for name in ancestor_leaves]
    
            #if the leaves of the parent node include an outgroup, throw out the gene
            print(ancestor_leaves_spec)
            if len(set(ancestor_leaves_spec) & set(outgroups)) > 0:
                print('Threw out ' + branch_gene + ' because outgroup gene between it and seed')
                leaf_subset.remove(branch_gene)    
                leaf_subset_spec = [name.split("_")[1] for name in leaf_subset]
                leaf_subset_spec_counter = Counter(leaf_subset_spec)
                
            if  (leaf_subset_spec_counter[WGH_branch]==0):
                print('Alert threw out both ' + WGH_branch + ' genes for seed: ' + gene)
    
    
# # #Extract those ones plus outgroups
# # leaf_names = tree.get_leaf_names()
# # 
# # print(tree)
# gene = ZT_genes[0]



#Warn if it throws out both Zygro genes


['YEAST', 'ZYGRO', 'YEAST', 'KLULA']
['DEKBR', 'YEAST', 'KLULA', 'DEKBR', 'YEAST', 'ZYGRO', 'YEAST', 'KLULA', 'CANAL', 'YARLI']
Threw out Phy0008N7U_KLULA because outgroup gene between it and seed


In [174]:

a =
a

'Phy000CXYH_YEAST'

In [148]:
leaf_subset

['Phy000CXYH_YEAST',
 'Phy000CXQ1_YEAST',
 'Phy00246YR_ZYGRO',
 'Phy0008PGK_KLULA']

In [11]:
#Load goi data

ohnologs_goi = pd.read_csv(data_processing_dir + os.path.normpath('20170817_klscpka/20170817_ohnologs_goi.csv'),index_col = 0)


In [197]:
#Add topology columns
sc_phylomedb_convert_topology = pd.merge(sc_phylomedb_convert, pd.DataFrame(trees['topologies']), how = 'left', left_on = 'phylomedb_name', right_index = True)
ohnologs_topology_gene1 = pd.merge(ohnologs_goi, sc_phylomedb_convert_topology, how = 'left', left_on = 'sc_genename_low', right_on = 'sc_genename' )
ohnologs_topology = pd.merge(ohnologs_topology_gene1, sc_phylomedb_convert_topology, how = 'left', left_on = 'sc_genename_high', right_on = 'sc_genename',suffixes = ['_low', '_high'])


In [155]:
columns_to_view = ['SC_common_name_low', 'SC_common_name_high','log2FoldChange_SC_low', 'log2FoldChange_SC_high', 'log2FoldChange_KL_low','topologies_low','topologies_high','threshold_group_KL','threshold_group_SC']

In [69]:
ohnologs_topology[(ohnologs_topology['topologies_low']=='missing KLULA protein') | (ohnologs_topology['topologies_high']=='missing KLULA protein')]


,Number,Ancestor,AA %id,Length Ratio,SC_common_name_YGOB_low,N SCer Orthologs_low,kl_genename_low,sc_genename_low,baseMean_KL_low,log2FoldChange_KL_low,...,threshold_group_SC,threshold_group_KL,included_go_terms_low,included_go_terms_high,phylomedb_name_low,sc_genename_low,topologies_low,phylomedb_name_high,sc_genename_high,topologies_high
2,30,Anc_1.212,44%,0.62,GLG2,2,KLLA0A05709g,YJL137C,91.242229,1.418622,...,up_flat,flat,"['carbohydrate metabolic process', 'generation...","['carbohydrate metabolic process', 'generation...",Phy000CY2W_YEAST,YJL137C,A,Phy000NLTT_YEAST,YKR058W,missing KLULA protein
10,111,Anc_2.256,41%,0.43,MSG5,2,KLLA0F03597g,YNL053W,149.447493,1.038979,...,up_flat,flat,NaN,NaN,Phy000CZN9_YEAST,YNL053W,missing KLULA protein,Phy000CWSQ_YEAST,YIL113W,B
18,148,Anc_2.598,29%,0.90,MSN2,2,KLLA0F26961g,YMR037C,234.389096,0.757359,...,flat_flat,flat,NaN,NaN,Phy000CZ64_YEAST,YMR037C,missing KLULA protein,Phy000CYGA_YEAST,YKL062W,missing KLULA protein
39,351,Anc_6.70,21%,0.82,PIG1,2,KLLA0D09240g,YLR273C,47.205327,1.335949,...,up_flat,flat,"['carbohydrate metabolic process', 'generation...","['carbohydrate metabolic process', 'generation...",Phy000CYV0_YEAST,YLR273C,missing KLULA protein,Phy000D017_YEAST,YOR178C,missing KLULA protein
42,381,Anc_6.256,31%,0.54,RGM1,2,KLLA0C16005g,YMR182C,99.590392,0.710576,...,up_flat,flat,NaN,NaN,Phy000CZAI_YEAST,YMR182C,missing KLULA protein,Phy000D093_YEAST,YPL230W,NaN
47,422,Anc_7.230,34%,0.98,PIG2,2,KLLA0F27533g,YIL045W,693.647965,1.811906,...,up_flat,flat,"['carbohydrate metabolic process', 'generation...","['carbohydrate metabolic process', 'generation...",Phy000CWUS_YEAST,YIL045W,missing ZYGRO and KLULA protein,Phy000CX2P_YEAST,YER054C,missing KLULA protein


In [198]:
#ohnologs_topology_sorted.loc[:,['topologies_low','topologies_high']]
list(set(list(ohnologs_topology['topologies_low'])+list(ohnologs_topology['topologies_high'])))


[nan,
 'C',
 'B',
 'under construction',
 'missing KLULA protein',
 'missing TORDC and KLULA protein',
 'A',
 'missing TORDC protein']

In [200]:
print(Counter(ohnologs_topology_sorted['topologies_low']))
print(Counter(ohnologs_topology_sorted['topologies_high']))

Counter({'A': 31, 'C': 14, 'B': 4, 'missing KLULA protein': 4, 'missing TORDC and KLULA protein': 4, 'under construction': 3, nan: 1})
Counter({'A': 26, 'C': 22, 'missing KLULA protein': 4, 'B': 2, 'under construction': 2, 'missing TORDC and KLULA protein': 2, nan: 2, 'missing TORDC protein': 1})


In [187]:
print(Counter(ohnologs_topology_sorted['topologies_low']))
print(Counter(ohnologs_topology_sorted['topologies_high']))

Counter({'A': 32, 'C': 14, 'B': 4, 'missing KLULA protein': 4, 'missing ZYGRO and KLULA protein': 4, 'under construction': 2, nan: 1})
Counter({'A': 25, 'C': 23, 'missing KLULA protein': 4, 'under construction': 3, 'B': 2, 'missing ZYGRO and KLULA protein': 2, nan: 2})


In [194]:
ZT_branch_seed

'TORDC'

In [199]:
#Visualize with a heatmap

ohnologs_topology_sorted = ohnologs_topology.sort_values(by = ['threshold_group_SC','log2FoldChange_KL_low'])

fc_values = ohnologs_topology_sorted.loc[:,['log2FoldChange_SC_low', 'log2FoldChange_SC_high', 'log2FoldChange_KL_low']]

value_dict = {'A':-3, 'B':3,'C':0,
              'under construction':1,
              'missing KLULA protein':2,
              'missing ' + ZT_branch_seed + ' and KLULA protein':-1,
              np.nan:-2,
              'missing ' + ZT_branch_seed + ' protein':-1.5
             }
topology_values = []

for topology_column in ['topologies_low','topologies_high']:
    topology_values.append([value_dict[topology] for topology in ohnologs_topology_sorted[topology_column]])



trace1 = pygo.Heatmap(z = np.array(fc_values),
                   x=['SC low FC', 'SC high FC', 'KL FC'],
                   y= ohnologs_topology_sorted['SC_common_name_low']+ ' / ' + ohnologs_topology_sorted['SC_common_name_high'])
trace2 = pygo.Heatmap(z = np.array(topology_values).transpose(),
                   x=['topology low', 'topology high'],
                   y= ohnologs_topology_sorted['SC_common_name_low']+ ' / ' + ohnologs_topology_sorted['SC_common_name_high'])


fig = pytools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

#fig['layout'].update(height=600, width=600, title='i <3 subplots')

# layout = pygo.Layout(
#     yaxis = {'title': 'Low Paralog / High Paralog'}
# )

#fig = pygo.Figure(data=data, layout=layout)
py.iplot(fig, filename='topology_heatmap')




This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



,alignment_method,score,tree,topologies
phylomedb_name,,,,
Phy000D070_YEAST,LG,-8846.39,"(((((((((Phy000NLV5_SACBA:0.0681393,Phy000CVIN...",A
Phy000CXYH_YEAST,LG,-7800.97,"(((((((((((((((((Phy004FJX0_NAUDC:0.333319,(Ph...",under construction
Phy000D068_YEAST,LG,-12821.60,"(((((((((((Phy000NQOY_SACCA:0.0408472,Phy004FI...",C
Phy000CVJD_YEAST,LG,-12768.60,"(((((((((((Phy000NQOY_SACCA:0.0403748,Phy004FI...",C
Phy000CXQ1_YEAST,LG,-7779.94,"(((((((((((((((((((Phy004FEEO_KAZAF:0.596488,(...",under construction
Phy000CVIN_YEAST,LG,-8822.98,"(((((((((Phy000NPZ4_SACBA:0.0366906,Phy000D070...",A


In [85]:
#look at tree topology for given sc common name
gois = ['GDH1','GDH3','CDC19','PYK2','MPC2','MPC3']

phylomedb_names = sc_phylomedb_convert[sc_phylomedb_convert['sc_genename'].isin(io_library.SC_orf_lookup_by_name(gois))].copy()

phylomedb_names['SC_common_name'] = io_library.SC_common_name_lookup(phylomedb_names['sc_genename'])

trees_subset = {}
for ind, row in phylomedb_names.iterrows():
    trees_subset[row['SC_common_name']]=trees.loc[row['phylomedb_name'],'tree']






In [1]:
columns_to_view = ['Gene 1', 'Gene Name 1', 'log2FoldChange_SC_gene1','Gene 2', 'Gene Name 2',  'log2FoldChange_SC_gene2', 'threshold_group_SC','kl_genename_gene1','log2FoldChange_KL_gene1','threshold_group_KL']
ohnologs_expression[ohnologs_expression['Gene Name 1'].isin(['SDC25','YDR134C','YGR273C'])].loc[:,columns_to_view]


NameError: name 'ohnologs_expression' is not defined

In [37]:
phylomedb_names = ['Phy000CZF5_YEAST', 'Phy000CX7L_YEAST', 'Phy000CZ7U_YEAST']
sc_phylomedb_convert_topology[sc_phylomedb_convert_topology['phylomedb_name'].isin(phylomedb_names)]

,phylomedb_name,sc_genename,topologies
288,Phy000CZ7U_YEAST,YMR096W,under construction
3578,Phy000CZF5_YEAST,YNL333W,under construction
3775,Phy000CX7L_YEAST,YFL059W,under construction


In [12]:
ohnologs_topology[ohnologs_topology['Gene Name low'].isin(['GDH1','GDH3'])]
#['Gene 1', 'Gene Name 1', 'log2FoldChange_SC_gene1','Gene 2', 'Gene Name 2',  'log2FoldChange_SC_gene2', 'threshold_group_SC','kl_genename_gene1','log2FoldChange_KL_gene1','threshold_group_KL']

,threshold_group_SC,threshold_group_KL,kl_genename_gene1,log2FoldChange_KL_gene1,Gene low,Gene Name low,log2FoldChange_SC_low,Gene high,Gene Name high,log2FoldChange_SC_high,phylomedb_name_low,sc_genename_low,topologies_low,phylomedb_name_high,sc_genename_high,topologies_high
379,down_flat,flat,KLLA0F00594g,-0.50328,YOR375C,GDH1,-3.521635,YAL062W,GDH3,2.46504,Phy000D070_YEAST,YOR375C,A,Phy000CVIN_YEAST,YAL062W,A


In [17]:
#filter to up_flat in SC and flat in KL.
ohnologs_topology_subset = ohnologs_topology[(ohnologs_topology['threshold_group_SC']=='up_flat') & (ohnologs_topology['threshold_group_KL']=='flat')]

ohnologs_topology_subset[ohnologs_topology_subset['Gene Name low'].isin(['CDC19','PYK2','MPC2','MPC3'])]

,threshold_group_SC,threshold_group_KL,kl_genename_gene1,log2FoldChange_KL_gene1,Gene low,Gene Name low,log2FoldChange_SC_low,Gene high,Gene Name high,log2FoldChange_SC_high,phylomedb_name_low,sc_genename_low,topologies_low,phylomedb_name_high,sc_genename_high,topologies_high
261,up_flat,flat,KLLA0A05500g,-1.789380,YHR162W,MPC2,-1.860001,YGR243W,FMP43,3.387496,Phy000CXYH_YEAST,YHR162W,under construction,Phy000CXQ1_YEAST,YGR243W,under construction
383,up_flat,flat,KLLA0F23397g,0.086733,YAL038W,CDC19,-0.908989,YOR347C,PYK2,2.544465,Phy000CVJD_YEAST,YAL038W,C,Phy000D068_YEAST,YOR347C,C


In [50]:
ohnologs[ohnologs['Gene 2']=='YCR075W-A']

,Number,Ancestor,Gene 1,Gene Name 1,Gene 2,Gene Name 2,AA %id,Length Ratio
396,397,Anc_6.348,YNR034W-A,YNR034W-A,YCR075W-A,YCR075W-A,18%,0.77


In [51]:
ohnologs_topology[ohnologs_topology['Gene Name low']=='YCR075W-A']

,threshold_group_SC,threshold_group_KL,kl_genename_gene1,log2FoldChange_KL_gene1,Gene low,Gene Name low,log2FoldChange_SC_low,Gene high,Gene Name high,log2FoldChange_SC_high,phylomedb_name_low,sc_genename_low,topologies_low,phylomedb_name_high,sc_genename_high,topologies_high
376,up_flat,flat,KLLA0F27390g,0.257658,YCR075W-A,YCR075W-A,0.542983,YNR034W-A,YNR034W-A,7.183301,Phy000CW2F_YEAST,YCR075W-A,C,Phy000CZPR_YEAST,YNR034W-A,under construction


In [33]:
Counter(ohnologs_topology_subset['topologies_low'])

Counter({'under construction': 40, nan: 3, 'C': 13, 'A': 22, 'B': 2})

In [34]:
Counter(ohnologs_topology_subset['topologies_high'])

Counter({'under construction': 40, 'C': 17, 'A': 17, 'B': 1, nan: 5})

In [171]:
fig_data

{'marker': {'opacity': 0.5},
 'mode': 'markers',
 'text': ['HSP150<br>under construction<br>PIR3<br>under construction',
  'GZF3<br>under construction<br>DAL80<br>under construction',
  'LSB3<br>nan<br>YSC84<br>C',
  'RCK1<br>under construction<br>RCK2<br>under construction',
  'IDP2<br>under construction<br>IDP3<br>under construction',
  'CYC7<br>C<br>CYC1<br>C',
  'YEF1<br>A<br>UTR1<br>A',
  'YJR054W<br>C<br>PRM6<br>C',
  'PTK2<br>A<br>PTK1<br>under construction',
  'ADY3<br>C<br>CNM67<br>under construction',
  'IGO2<br>A<br>IGO1<br>A',
  'YPT53<br>under construction<br>VPS21<br>under construction',
  'SDP1<br>B<br>MSG5<br>under construction',
  'COX5A<br>under construction<br>COX5B<br>under construction',
  'NDE2<br>under construction<br>NDE1<br>under construction',
  'YLR031W<br>under construction<br>YMR124W<br>under construction',
  'SDH3<br>B<br>SHH3<br>C',
  'YKL133C<br>C<br>MGR3<br>A',
  'DGR2<br>A<br>YMR102C<br>A',
  'YKL091C<br>under construction<br>SEC14<br>under constructio

In [192]:
hover_text = []
for ind, row in ohnologs_topology_subset.iterrows():
    #(row['Gene Name 1'],row['topologies_gene1'],row['Gene Name 2'], row['topologies_gene2'])
    hover_text.append('{}<br>{}<br>{}<br>{}'.format(row['Gene Name 1'],row['topologies_gene1'],row['Gene Name 2'], row['topologies_gene2']))

#titles = {1: 'One S.Cer ortholog ', 2: 'Two S.Cer orthologs '}
#colors = {1: 'grey', 2: 'red'}

trace = pygo.Scatter(
            x= ohnologs_topology_subset['log2FoldChange_SC_gene1'], #[1,2,3,4],
            y= ohnologs_topology_subset['log2FoldChange_SC_gene2'], #[1.1,2,3.5,7],
            mode= 'markers',
            text= hover_text #['a','b','c','d']
            )

layout = pygo.Layout(
    title= 'topology for ohnologs',
    hovermode= 'closest',
    xaxis= {'title':'gene 1 fold change'},
    yaxis= {'title':'gene 2 fold change'}
    )
    
fig= pygo.Figure(data=[trace], layout=layout)


# N = 1000
# random_x = np.random.randn(N)
# random_y = np.random.randn(N)

# # Create a trace
# trace = pygo.Scatter(
#     x = random_x,
#     y = random_y,
#     mode = 'markers',
#     text = random_x
# )

# data = [trace]

# layout = pygo.Layout(
#     title= 'Stats of USA States',
#     hovermode= 'closest'
#     )

# fig= pygo.Figure(data=data, layout=layout)

# Plot and embed in ipython notebook!
#py.iplot(data, filename='basic-scatter')

#py.iplot(fig, filename='SC_foldchange_topology_overlay')
py.iplot(fig, filename='SC_foldchange_topology_overlay')


# fig_data = {
#   			'x': [1,2,3,4.5],    #list(ohnologs_topology_subset['log2FoldChange_SC_gene1']), 
#         	'y': [2,3,4,5.7],    #list(ohnologs_topology_subset['log2FoldChange_SC_gene2']) 
#             'mode': 'markers'
#             } #, 
#         	'text': hover_text, 
#         	'mode': 'markers', 
#             'marker': {
#                 'opacity': 0.5,
#                 #'color': colors[NN]
#                 }
#             }
        	#'name': } 

# fig = {
#     'data': fig_data
#     'layout': {
#                 'xaxis': {'title': 'Gene 1 fold change'},
#                 'yaxis': {'title': 'Gene 2 fold change'}
#                }
       





In [152]:
#filter to genes with separated expression
foldchange_separation = 3.5
ohnologs_topology_separated = ohnologs_topology[abs(ohnologs_topology['log2FoldChange_SC_gene1'] - ohnologs_topology['log2FoldChange_SC_gene2']) > foldchange_separation]


columns_to_view = ['Gene 1', 'Gene Name 1', 'log2FoldChange_SC_gene1','topologies_gene1', 'Gene 2', 'Gene Name 2',  'log2FoldChange_SC_gene2', 'topologies_gene2','threshold_group_SC','log2FoldChange_KL_gene1','threshold_group_KL']
#['Gene 1', 'Gene Name 1', 'kl_genename_gene1', 'Gene 2', 'Gene Name 2', 'kl_genename_gene2', 'log2FoldChange_SC_gene1', 'log2FoldChange_SC_gene2', 'threshold_group_SC','log2FoldChange_KL_gene1','threshold_group_KL', 'topologies_gene1', 'topologies_gene2']
ohnologs_topology_separated.loc[:,columns_to_view]


,Gene 1,Gene Name 1,log2FoldChange_SC_gene1,topologies_gene1,Gene 2,Gene Name 2,log2FoldChange_SC_gene2,topologies_gene2,threshold_group_SC,log2FoldChange_KL_gene1,threshold_group_KL
0,YCL051W,LRE1,1.006787,C,YDR528W,HLR1,-3.262097,under construction,down_flat,-0.749493,flat
2,YDR524C-B,YDR524C-B,-2.236981,NaN,YCL048W-A,YCL048W-A,7.830939,NaN,up_down,-0.149862,flat
6,YCL037C,SRO9,-4.754452,under construction,YDR515W,SLF1,0.379591,under construction,down_flat,-4.762072,down
19,YCR034W,FEN1,-2.858003,under construction,YJL196C,ELO1,1.016717,under construction,down_flat,-0.688902,flat
24,YJL164C,TPK1,2.844732,under construction,YKL166C,TPK3,-2.613249,under construction,up_down,0.774367,flat
26,YJL159W,HSP150,0.711945,under construction,YKL163W,PIR3,6.987255,under construction,up_flat,0.254423,flat
35,YJL110C,GZF3,-1.197971,under construction,YKR034W,DAL80,2.719467,under construction,up_flat,-0.648261,flat
42,YJL079C,PRY1,1.883493,NaN,YKR013W,PRY2,-2.156467,NaN,down_flat,4.629306,up
50,YFR017C,IGD1,5.532256,under construction,YOL024W,YOL024W,1.159039,under construction,up_flat,2.714056,up
56,YLR249W,YEF3,-5.379224,under construction,YNL014W,HEF3,3.247235,under construction,up_down,-3.102575,down


In [8]:
#Load Ohnolog data

#All WGD paralogs
ohnologs = pd.read_csv(data_processing_dir + os.path.normpath("ortholog_files_YGOB/ohnologs.csv"))
kl_sc_PKA_data_paralogs = kl_sc_PKA_data[kl_sc_PKA_data["N SCer Orthologs"]==2]

#build a dataframe that has all SC paralogs paired up with log fold change and mean expression columns.  Also has K.Lactis log fold change and mean expression columns. 
ohnologs_expression_gene1 = pd.merge(ohnologs, kl_sc_PKA_data_paralogs, how = 'inner', left_on = 'Gene 1', right_on = 'sc_genename')
ohnologs_expression = pd.merge(ohnologs_expression_gene1, kl_sc_PKA_data_paralogs, how = 'inner', left_on = 'Gene 2', right_on = 'sc_genename', suffixes = ['_gene1', '_gene2'])

#Assign sc threshold groups
high_threshold_SC = 2.5
low_threshold_SC = -high_threshold_SC
ohnologs_expression = ohnologs_expression.assign(threshold_group_SC = lambda x: io_library.threshold_group_SC_series(x.log2FoldChange_SC_gene1,x.log2FoldChange_SC_gene2,high_threshold_SC, low_threshold_SC))

#Assign kl threshold groups
high_threshold_KL = high_threshold_SC
low_threshold_KL = low_threshold_SC
ohnologs_expression = ohnologs_expression.assign(threshold_group_KL = lambda x: io_library.threshold_group_KL_series(x.log2FoldChange_KL_gene1, high_threshold_KL, low_threshold_KL))

ohnologs_expression



,Number,Ancestor,Gene 1,Gene Name 1,Gene 2,Gene Name 2,AA %id,Length Ratio,N SCer Orthologs_gene1,kl_genename_gene1,...,rlog_AS_yesNMPP1_YPD_t40_mean_SC_gene2,rlog_AS_noNMPP1_YPD_t40_var_SC_gene2,rlog_AS_yesNMPP1_YPD_t40_var_SC_gene2,raw_AS_noNMPP1_YPD_t40_mean_SC_gene2,raw_AS_yesNMPP1_YPD_t40_mean_SC_gene2,raw_AS_noNMPP1_YPD_t40_var_SC_gene2,raw_AS_yesNMPP1_YPD_t40_var_SC_gene2,SC_common_name_gene2,threshold_group_SC,threshold_group_KL
0,1,Anc_1.15,YCL051W,LRE1,YDR528W,HLR1,25%,0.73,2,KLLA0C00803g,...,1.710101,0.017602,0.226643,1.205752,0.396789,0.003053,0.147992,HLR1,down_flat,flat
1,2,Anc_1.17,YCL050C,APA1,YDR530C,APA2,60%,0.99,2,KLLA0C00847g,...,6.096006,0.099664,0.151030,1.403782,2.040858,0.013446,0.017233,APA2,flat_flat,flat
2,3,Anc_1.22,YDR524C-B,YDR524C-B,YCL048W-A,YCL048W-A,48%,0.84,2,KLLA0C00957g,...,7.674560,0.031475,4.004451,0.531831,2.594901,0.012351,0.485654,YCL048W-A,up_flat,flat
3,4,Anc_1.25,YCL048W,SPS22,YDR522C,SPS2,42%,0.92,2,KLLA0C01001g,...,-0.444655,0.077548,0.070064,0.190185,0.123776,0.029958,0.045961,SPS2,flat_flat,up
4,5,Anc_1.31,YCL043C,PDI1,YDR518W,EUG1,39%,0.99,2,KLLA0C01111g,...,6.704994,0.004124,0.003458,2.013407,2.138036,0.002916,0.001584,EUG1,flat_flat,flat
5,6,Anc_1.33,YCL040W,GLK1,YDR516C,EMI2,73%,1.00,2,KLLA0C01155g,...,13.074401,0.053348,0.055359,3.271367,4.148126,0.017298,0.005234,EMI2,up_flat,flat
6,7,Anc_1.36,YCL037C,SRO9,YDR515W,SLF1,25%,0.97,2,KLLA0C01210g,...,4.996927,0.004003,0.054774,1.440944,1.674896,0.000485,0.008530,SLF1,down_flat,down
7,8,Anc_1.37,YCL036W,GFD2,YDR514C,YDR514C,37%,0.85,2,KLLA0C01254g,...,5.521014,0.137600,0.114629,2.253541,1.749123,0.008742,0.019621,YDR514C,down_flat,flat
8,9,Anc_1.38,YCL035C,GRX1,YDR513W,GRX2,62%,0.77,2,KLLA0C01298g,...,9.220533,1.104274,1.132657,1.923534,2.988741,0.128166,0.150249,GRX2,up_up,flat
9,10,Anc_1.50,YCL025C,AGP1,YDR508C,GNP1,67%,0.95,2,KLLA0C01606g,...,8.349347,0.071366,0.120381,3.064549,2.566909,0.003640,0.022230,GNP1,flat_flat,flat


In [9]:
#Convert ohnologs_expression to a sorted version. 

#loop through each item. 


columns_by_gene_no = {'gene1': ['Gene 1', 'Gene Name 1', 'log2FoldChange_SC_gene1'], 'gene2': ['Gene 2', 'Gene Name 2',  'log2FoldChange_SC_gene2']}

#make new columns: 
new_column_list = ['Gene low', 'Gene Name low', 'log2FoldChange_SC_low', 'Gene high', 'Gene Name high', 'log2FoldChange_SC_high']

new_columns_data = []

for index, row in ohnologs_expression.iterrows():
    #Decide if gene1 or gene2 is low expression
    if row['log2FoldChange_SC_gene1']<row['log2FoldChange_SC_gene2']:
        low_gene = 'gene1'
        high_gene = 'gene2'
    elif  row['log2FoldChange_SC_gene1']>row['log2FoldChange_SC_gene2']:
        low_gene = 'gene2'
        high_gene = 'gene1'
    else:
        print('problems with {} and {}'.format(row['Gene Name 1'],row['Gene Name 2']))
    
    new_columns_data.append([row[column] for column in columns_by_gene_no[low_gene]] + [row[column] for column in columns_by_gene_no[high_gene]])

#existing columns to keep
columns_to_keep = ['threshold_group_SC','threshold_group_KL', 'kl_genename_gene1','log2FoldChange_KL_gene1']
ohnologs_expression_sorted = ohnologs_expression.loc[:,columns_to_keep]

#this seems to be looping through inefficiently
for ind,new_column_name in enumerate(new_column_list):
    ohnologs_expression_sorted[new_column_name] = [row[ind] for row in new_columns_data]

ohnologs_expression_sorted

# #['Gene 1', 'Gene Name 1', 'log2FoldChange_SC_gene1','Gene 2', 'Gene Name 2',  'log2FoldChange_SC_gene2', 'threshold_group_SC','kl_genename_gene1','log2FoldChange_KL_gene1','threshold_group_KL']
# ohnologs_expression.columns

problems with SDC25 and CDC25
problems with YGR273C and YMR295C
problems with YDR134C and CCW12


,threshold_group_SC,threshold_group_KL,kl_genename_gene1,log2FoldChange_KL_gene1,Gene low,Gene Name low,log2FoldChange_SC_low,Gene high,Gene Name high,log2FoldChange_SC_high
0,down_flat,flat,KLLA0C00803g,-0.749493,YDR528W,HLR1,-3.262097,YCL051W,LRE1,1.006787
1,flat_flat,flat,KLLA0C00847g,0.725481,YCL050C,APA1,-0.184330,YDR530C,APA2,1.719713
2,up_flat,flat,KLLA0C00957g,-0.149862,YDR524C-B,YDR524C-B,-2.236981,YCL048W-A,YCL048W-A,7.830939
3,flat_flat,up,KLLA0C01001g,2.658001,YDR522C,SPS2,-1.076672,YCL048W,SPS22,2.212668
4,flat_flat,flat,KLLA0C01111g,1.039729,YDR518W,EUG1,-0.030605,YCL043C,PDI1,-0.003622
5,up_flat,flat,KLLA0C01155g,2.178913,YDR516C,EMI2,2.464130,YCL040W,GLK1,4.160390
6,down_flat,down,KLLA0C01210g,-4.762072,YCL037C,SRO9,-4.754452,YDR515W,SLF1,0.379591
7,down_flat,flat,KLLA0C01254g,-0.748394,YCL036W,GFD2,-3.913143,YDR514C,YDR514C,-2.131266
8,up_up,flat,KLLA0C01298g,1.719177,YCL035C,GRX1,2.965390,YDR513W,GRX2,3.051427
9,flat_flat,flat,KLLA0C01606g,-2.184941,YDR508C,GNP1,-2.082446,YCL025C,AGP1,0.734168
